In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers.experimental import SGD
from sklearn.model_selection import train_test_split,GridSearchCV
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report


In [33]:
datas = pd.read_csv("/home/khalifardy/Dokumen/ruang_kerja/code/kuliah/ML/Assignment_CLO1_project_base/prediksi_dropout/pre_processing_data/data_eda.csv")
total = datas["Target"].value_counts()
dictio ={
    "graduate":total[0]/len(datas),
    'dropout':total[1]/len(datas),
    'enrolled':total[2]/len(datas)
}
dictio



/tmp/ipykernel_3399/3771920146.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "graduate":total[0]/len(datas),
/tmp/ipykernel_3399/3771920146.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'dropout':total[1]/len(datas),
/tmp/ipykernel_3399/3771920146.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'enrolled':total[2]/len(datas)


{'graduate': 0.47342485040478705,
 'dropout': 0.2653995072157691,
 'enrolled': 0.26117564237944385}

In [34]:
datas["Target"] = datas["Target"].map( {
    "Dropout":0,
    "Graduate":1,
    "Enrolled":2
})

In [35]:
X = datas[datas.columns[:-1]]
Y = datas[datas.columns[-1]]

Y_encode = to_categorical(Y,num_classes=3)

In [36]:
x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size=0.2,stratify=Y, random_state=10)

In [37]:
X_train = x_train.to_numpy()
y_train = np.array(y_train)
X = X.to_numpy()
Y_encode = np.array(Y_encode)

In [38]:
dim = len(X)
models = tf.keras.Sequential(
    [tf.keras.layers.Dense(units=dim, input_shape=(len(x_train.columns),)),
    tf.keras.layers.Dense(512, activation='leaky_relu'),
    tf.keras.layers.Dense(264, activation='leaky_relu'),
    tf.keras.layers.Dense(128, activation='leaky_relu'),
    tf.keras.layers.Dense(64, activation='leaky_relu'),
    tf.keras.layers.Dense(32, activation='leaky_relu'),
    tf.keras.layers.Dense(16, activation='leaky_relu'),
    tf.keras.layers.Dense(8, activation='leaky_relu'),
    tf.keras.layers.Dense(4, activation='leaky_relu'),
    tf.keras.layers.Dense(2, activation='leaky_relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation='softmax')]
)
models.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 2841)              59661     
                                                                 
 dense_34 (Dense)            (None, 512)               1455104   
                                                                 
 dense_35 (Dense)            (None, 264)               135432    
                                                                 
 dense_36 (Dense)            (None, 128)               33920     
                                                                 
 dense_37 (Dense)            (None, 64)                8256      
                                                                 
 dense_38 (Dense)            (None, 32)                2080      
                                                                 
 dense_39 (Dense)            (None, 16)               

In [41]:
epoch = 200
models.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.001),
    metrics=['accuracy']
)
models.fit(X,Y_encode,validation_split=0.33,epochs=epoch,verbose=2)

Epoch 1/200
60/60 - 3s - loss: 0.8204 - accuracy: 0.5870 - val_loss: 5.0858 - val_accuracy: 0.1269 - 3s/epoch - 58ms/step
Epoch 2/200
60/60 - 2s - loss: 0.5849 - accuracy: 0.6868 - val_loss: 4.5772 - val_accuracy: 0.1226 - 2s/epoch - 27ms/step
Epoch 3/200
60/60 - 2s - loss: 0.5800 - accuracy: 0.7010 - val_loss: 4.4897 - val_accuracy: 0.1407 - 2s/epoch - 32ms/step
Epoch 4/200
60/60 - 1s - loss: 0.5513 - accuracy: 0.7173 - val_loss: 4.6569 - val_accuracy: 0.0288 - 1s/epoch - 25ms/step
Epoch 5/200
60/60 - 2s - loss: 0.5471 - accuracy: 0.7168 - val_loss: 4.4856 - val_accuracy: 0.1365 - 2s/epoch - 32ms/step
Epoch 6/200
60/60 - 1s - loss: 0.5348 - accuracy: 0.7252 - val_loss: 5.7887 - val_accuracy: 0.0608 - 1s/epoch - 25ms/step
Epoch 7/200
60/60 - 1s - loss: 0.5233 - accuracy: 0.7415 - val_loss: 9.3816 - val_accuracy: 0.0810 - 1s/epoch - 25ms/step
Epoch 8/200
60/60 - 2s - loss: 0.5151 - accuracy: 0.7535 - val_loss: 5.3326 - val_accuracy: 0.0725 - 2s/epoch - 27ms/step
Epoch 9/200
60/60 - 1s -

In [47]:
datas["Target"].value_counts()[0]/len(datas)

0.35063492063492063

In [44]:
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=22)

# Define and compile your model once, outside the loop
models.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.001),
    metrics=['accuracy']
)

for index_train, index_test in skf.split(X, Y_encode):
    train_x, test_x = X[index_train], X[index_test]
    train_y, test_y = Y_encode[index_train], Y_encode[index_test]

  # Replace create_model() with code to create your model

    # Fit the model within the loop for each fold
    models.fit(train_x, train_y, epochs=200, verbose=0)
    scores = models.evaluate(test_x, test_y, verbose=0)

    print("Validation accuracy: {:.2f}%".format(scores[1] * 100))


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multilabel-indicator' instead.

In [1]:
class KnearestNeighbors:
    def __init__(self, k=1):
        self.k = k

    def euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1-x2)**2))

    def manhattan_distance(self, x1, x2):
        return np.sum(np.absolute(x1-x2))

    def list_distance(self, tipe, xtrain, xtest, posisi):
        distance = []
        for i in range(len(xtrain)):
            if tipe == "euclidean":
                distance.append(self.euclidean_distance(
                    xtrain.loc[i], xtest.loc[posisi]))
            elif tipe == "manhattan":
                distance.append(self.manhattan_distance(
                    xtrain.loc[i], xtest.loc[posisi]))

        return distance

    def predict(self, tipe, xtrain, xtest, posisi, ytrain, k=None):

        if k is None:
            k = self.k
        distance = self.list_distance(
            tipe, xtrain, xtest, posisi)
        data_ = xtrain.copy()
        data_["distance"] = distance
        data_["label"] = ytrain

        data_ = data_.sort_values(by="distance")
        y_pred = data_[:k].label.mode()

        return y_pred[0]

    def accuracy(self, y_pred, y_test):
        benar = 0
        for i in range(len(y_pred)):
            if y_pred[i] == y_test[i]:
                benar += 1

        return benar/len(y_test)

    def evaluasi(self, tipe, train, test, label, k=None, ):
        if k is None:
            k = self.k

        xtrain, ytrain = train.drop(label, axis=1), train[label]
        xtest, ytest = test.drop(label, axis=1), test[label]

        ypred = []
        for i in range(len(xtest)):
            ypred.append(self.predict(tipe, xtrain, xtest, i, ytrain, k))

        return self.accuracy(ypred, ytest)

    def get_average_accuracy(self, tipe, list_data, label, k=None, cetak=False):
        if k is None:
            k = self.k

        akurasi = []

        for i in range(len(list_data)):
            train, test = list_data[i]
            akurasi.append(self.evaluasi(tipe, train, test, label, k))

        if cetak:
            print("Untuk k : {} , Rata-rata akurasi:{}".format(k,
                  sum(akurasi)/len(akurasi)))
        else:
            return sum(akurasi)/len(akurasi)

    def plot_evaluasi_k(self, tipe, list_data, label, start, end, step=1):

        average_accuracy = []
        k_list = []
        for i in range(start, end, step):
            k_list.append(i)
            average_accuracy.append(
                self.get_average_accuracy(tipe, list_data, label, i))

        plt.plot(k_list, average_accuracy)
        plt.title("evaluasi K {}-{}".format(start, end-1))
        plt.xlabel("k value")
        plt.ylabel("accuracy")
        plt.show()


In [2]:
def split_data(data, traininggPercentage, location, shuffle=False):
    lengthTraining = int(len(data)*traininggPercentage/100)

    if shuffle:
        data = data.sample(frac=1).reset_index(drop=True)
    train = []
    validation = []

    if (location == "left"):
        train, validation = data.iloc[:lengthTraining].reset_index(
            drop=True), data.iloc[lengthTraining:].reset_index(drop=True)
    elif (location == "right"):
        validation, train = data.iloc[:abs(lengthTraining-len(data))].reset_index(
            drop=True), data.iloc[abs(lengthTraining-len(data)):].reset_index(drop=True)
    elif (location == 'middle'):
        train = data.iloc[int(abs(lengthTraining-len(data))/2):len(data)-int(abs(lengthTraining-len(data))/2)].reset_index(
            drop=True)
        validation = pd.concat([data.iloc[:int(abs(lengthTraining-len(data))/2)],
                               data.iloc[len(data)-int(abs(lengthTraining-len(data))/2):]]).reset_index(
            drop=True)

    return train, validation

In [6]:
train,validation = split_data(datas,80,"middle",True)

In [7]:
knn = KnearestNeighbors()

In [8]:
akurasi_euclid = knn.evaluasi("euclidean", train, validation, "Target", k=15)

KeyboardInterrupt: 